In [1]:
import pandas as pd


In [ ]:
def string_to_list(string):
    # Remove leading and trailing whitespace
    string = string.strip()
    
    # Check if the string starts and ends with '[' and ']'
    if string[0] != '[' or string[-1] != ']':
        raise ValueError("Input string is not formatted as a list")
    
    # Remove '[' and ']' from the string
    string = string[1:-1]
    
    # Split the string by commas
    elements = string.split(',')
    
    # Initialize an empty list to store the converted elements
    result = []
    
    # Convert each element to its appropriate type and add it to the result list
    for element in elements:
        # Remove leading and trailing whitespace
        element = element.strip()
        
        # Check if the element is a string or a number
        if element.startswith("'") and element.endswith("'"):
            # If the element is a string, remove the quotes and append it to the result list
            result.append(element[1:-1])
      
    
    return result


In [11]:
users = pd.read_csv("users.csv")
users["ingredients"] = users["ingredients"].apply(string_to_list)

In [13]:
def format_ner(row):
    text = row['sentences']
    entities = []
    start_pos = 0
    for ingredient in row['ingredients']:
        start = text.find(ingredient)
        if start != -1:
            end = start + len(ingredient)
            entities.append((start, end, "INGREDIENT"))
    return (text, {"entities": entities})

# Apply the function to each row and create the "NER" column
users['NER'] = users.apply(format_ner, axis=1)

users.head()

,Unnamed: 0,sentences,ingredients,NER
0,0,"What can I cook with chicken, broccoli, and rice?","[chicken, broccoli, rice]","(What can I cook with chicken, broccoli, and r..."
1,1,"I have potatoes, carrots, and celery. What rec...","[potatoes, carrots, celery]","(I have potatoes, carrots, and celery. What re..."
2,2,"What dishes can I prepare with beef, bell pepp...","[beef, bell peppers, mushrooms]","(What dishes can I prepare with beef, bell pep..."
3,3,"What can I make with pasta, spinach, and cheese?","[pasta, spinach, cheese]","(What can I make with pasta, spinach, and chee..."
4,4,"What's a good vegetarian dish with lentils, to...","[lentils, tomatoes, onions]","(What's a good vegetarian dish with lentils, t..."


In [14]:
users["NER"][0]

('What can I cook with chicken, broccoli, and rice?',
 {'entities': [(21, 28, 'INGREDIENT'),
   (30, 38, 'INGREDIENT'),
   (44, 48, 'INGREDIENT')]})

In [15]:
import spacy
import random
from spacy.training.example import Example

# Load the blank English model
nlp = spacy.blank("en")

# Add NER pipeline
ner = nlp.add_pipe("ner")

# Add labels for ingredients
ner.add_label("INGREDIENT")

# Example training data
TRAIN_DATA = users["NER"][:int(0.8*len(users["NER"]))]

# Start training
nlp.begin_training()

# Train the model
for itn in range(3):  # Train for 10 iterations
    random.shuffle(TRAIN_DATA)
    losses = {}

    for text, annotations in TRAIN_DATA:

        example = Example.from_dict(nlp.make_doc(text), annotations)
        nlp.update([example], losses=losses)

    print(losses)

# Save the model to disk
nlp.to_disk("ner_model")

# Load the saved model
# nlp = spacy.load("ner_model")

# Test the trained model
test_text = "Cook spaghetti with tomato sauce"
doc = nlp(test_text)
print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])

{'ner': 179.3669997592461}
{'ner': 0.0001513773050299827}
{'ner': 1.1140925520542612e-07}
Entities: [('tomato sauce', 'INGREDIENT')]


In [17]:
for i in users["sentences"][int(0.8*len(users["NER"])):]:
    print(i, nlp(i).ents)
    

What's a good recipe for salmon, asparagus, and lemon pepper? (salmon, asparagus, lemon pepper)
What can I cook with chicken, broccoli, and garlic? (chicken, broccoli, garlic)
What's a good vegetarian dish with quinoa, black beans, and avocado? (quinoa, black beans, avocado)
What can I make with ground beef, potatoes, and carrots? (ground beef, potatoes, carrots)
What desserts can I make with raspberries, whipped cream, and shortcake? (raspberries, whipped cream, shortcake)
What can I cook with shrimp, garlic, and tomato sauce? (shrimp, garlic, tomato sauce)
I have pork tenderloin, sweet potatoes, and carrots. What's for dinner? (pork tenderloin, sweet potatoes, carrots)
What's a simple recipe for chicken, rice, and broccoli? (chicken, rice, broccoli)
What can I make with ground turkey, black beans, and corn tortillas? (ground turkey, black beans, corn tortillas)
What recipes can I make with eggplant, tomatoes, and ricotta cheese? (eggplant, tomatoes, ricotta)
What can I cook with tila

In [18]:
inp = input("What is your imput : ")
print(nlp(inp).ents)


(chocolate, butter, milk)
